In [1]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

# !pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

import numpy as np
import torch
import os

# print("Torch version:", torch.__version__)
# os.kill(os.getpid(), 9)
#Your notebook process will restart after these installs

CUDA version: 11.1


In [2]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 222, done.
remote: Total 222 (delta 0), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (222/222), 8.91 MiB | 30.12 MiB/s, done.
Resolving deltas: 100% (113/113), done.
/content/CLIP


In [3]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="clip", notebook="roboflow-clip")

     |████████████████████████████████| 145 kB 5.1 MB/s 
     |████████████████████████████████| 178 kB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.7 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 138 kB 71.1 MB/s 
     |████████████████████████████████| 596 kB 52.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
upload and label your dataset, and get an API KEY here: https://app.roboflow

In [4]:
# download classification data
from roboflow import Roboflow
rf = Roboflow(api_key="mkitKptEErYUTmUh8DjM")
project = rf.workspace().project("flowers")
dataset = project.version("1").download("clip")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Flowers-1 in clip:: 100%|██████████| 1827/1827 [00:01<00:00, 990.78it/s]


In [5]:
dataset.location

'/content/CLIP/Flowers-1'

In [6]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('/content/CLIP/tests/')
# class_names.remove('_tokenization.txt')
class_names

['test_consistency.py']

In [7]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat {dataset.location}/test/_tokenization.txt

cat: /content/CLIP/Flowers-1/test/_tokenization.txt: No such file or directory


In [8]:
#edit your prompts as you see fit here, be sure the classes are in teh same order as above
%%writefile /content/CLIP/Flowers-1/train/_tokenization.txt
The paper sign in rock paper scissors
The rock sign in rock paper scissors
The scissors sign in rock paper scissors

Overwriting /content/CLIP/Flowers-1/train/_tokenization.txt


In [9]:
candidate_captions = []
with open('/content/CLIP/Flowers-1/train/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [12]:
!pip install ftfy

     |████████████████████████████████| 53 kB 1.6 MB/s 


In [13]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob(dataset.location + '/test/' + cls + '/*.jpg')
    for img in test_imgs:
        #print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            #print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    # print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
# print('accuracy on all is : ' + str(sum(correct)/len(correct)))

100%|███████████████████████████████████████| 338M/338M [00:09<00:00, 38.2MiB/s]
